In [1]:
#libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
import netCDF4
import xarray as xr

## Inputs


Dimensions specified below

$\textit{m}$: length of the measurement vector

$\textit{n}$: length of the state (parameter) vector

$\textit{r}$: length of the model error vector


To perform this ICA, you'll need a few things:

1. Jacobian matrix, defined as the $ K_{j,i}(\textbf{x}) = \frac{\partial F_i (\textbf{x})}{\partial x_j}$ where $\textit{i}$ has length $\textit{m}$ and $\textit{j}$ has length $\textit{n}$. $F(\textbf{x})$ is the forward model at the state defined by $\textbf{x}$.

    jac [n x m]
    
2. Model error Jacobian matrix, defined as the $ K_{b,u,i}(\textbf{x}) = \frac{\partial F_i (\textbf{x})}{\partial b_u}$ where $\textit{i}$ has length $\textit{m}$ and $\textit{u}$ has length $\textit{r}$. $F(\textbf{x})$ is the forward model at the state defined by $\textbf{x}$. Note there are options in the Rodgers function to omit consideration of model errors if these are not known.

    jac_me [r x m]
    
3. Measurement error coveriance matrix, $S_{\epsilon}$. This can be specificed as either an $\textit{m}$ length vector of sigma squared measurement uncertainties or an $\textit{m} x \textit{m}$ full covariance matrix. In the former case, it is assumed that measurement uncertainty is uncorrelated. 

    err [m x m] or [m]   

4. Model error coveriance matrix, $S_{b}$. This can be specificed as either an $\textit{r}$ length vector of sigma squared measurement uncertainties or an $\textit{r} x \textit{r}$ full covariance matrix. In the former case, it is assumed that measurement uncertainty is uncorrelated. Note there are options below to omit consideration of model errors if these are not known.

    err_me [m x m] or [m]   
    
5. A priori error coveriance matrix, $S_{a}$. This can be specificed as either an $\textit{n}$ length vector of sigma squared measurement uncertainties or an $\textit{n} x \textit{n}$ full covariance matrix. In the former case, it is assumed that measurement uncertainty is uncorrelated. 

    ap [n x n] or [n]   
    
6. If calculating the detection probability: the value of the parameter in question

    mu [scalar]
    
        

# Outputs


### from rodgers()
1. Error covariance matrix, $\hat{S}$.

    S_hat [n x n]
    
2. Shannon Information Content, $SIC$.

    SIC [scalar]
    
3. Averaging kernel matrix, $A$. 

    AvgK [n x n]

4. Degrees of Freedom for Signal, $DFS$. 

    DFS [scalar]  

    
### from detect_prob()
1. Probability of detection, $P_d$. Note this requires inputs mu (parameter value in question) and sigma ($\sqrt{\hat{S}_{p,p}}$ where $p$ is the parameter index.)

    Pd [scalar]
    
    Pd_pcnt_str [percentage as string]


## from Amir:
    Relative:
σ_sys = np.array([0.01587401, 0.01244266, 0.00938955, 0.01092051, 0.00302288,
                  0.00544271, 0.00999068, 0.01467843, 0.0080387, 0.00944394,
                  0.0193447, 0.0224503, 0.02386379])

Absolute:
σ_sys = np.array([2.80453033e-03, 1.52152435e-03, 1.04028473e-03, 9.98138237e-04,
                  1.12356970e-04, 2.68196657e-04, 4.69710954e-04, 4.32004600e-04,
                  2.49862337e-04, 2.53730871e-04, 8.83851738e-05, 1.47447273e-04,
                  1.57220696e-04])

This is based on real modisa data using our OE algorithm, where we use the full spectrum to optimize things, which means that these values work better when you use all bands in MODIS. The systematic uncertainty dominates the random noise, so I feel comfortable ignoring the random component.

In [2]:
#function to calculate the parameter error covariance matrix.

#input Jacobian, K, [n x m], error covariance matrix Se, [m x m] and a priori matrix Sa, [n x n]
#jac_me and me are associated with model uncertainty - that parameterized uncertainty and its jacobian
def rodgers(jac, err, ap, model_error={}, model_error_jacobian={}): 
    #todo
    # consider microplastic simulation jacobian in the following manner: Se' = Se + Kb Sb Kbt, where 
    #  Se is the same as above, Sb is the microplastic parameter uncertainty, Kb the microplastic jacobian
    
        #check if error covariance matrix is square, or just diagonal values. If latter make full matrix
    if err.ndim == 1:
        ln=np.shape(err)
        err2d = np.zeros((ln[0], ln[0]))
        np.fill_diagonal(err2d, err)
        err=err2d

        #check if a priori covariance matrix is square, or just diagonal values. If latter make full matrix
    if ap.ndim == 1:
        ln=np.shape(ap)
        ap2d = np.zeros((ln[0], ln[0]))
        np.fill_diagonal(ap2d, ap)
        ap=ap2d        
            
        #section to verify compatable dimensions ------------------------------------------------------
    sh_jac = np.shape(jac)
    sh_err = np.shape(err)
    sh_ap = np.shape(ap)
    
    n_dim = sh_jac[0]
    m_dim = sh_jac[1]
    
    if not((sh_err[0] == sh_err[1]) and (sh_ap[0] == sh_ap[1])):
        print('ERROR: error covariance matrix or a priori matrix are not square')
        print('Error covariance matrix dimensions')
        print(sh_err)
        print('A priori matrix dimensions')
        print(sh_ap)
        return -1, -1, -1, -1
    
    if not(sh_jac[0] == sh_ap[0]):
        print('ERROR: n dimensions inconsistent, should be Jacobian [n x m]; a priori [n x n]')
        print('Jacobian matrix dimensions')
        print(sh_jac)
        print('A priori matrix dimensions')
        print(sh_ap)
        return -1, -1, -1, -1
    
    if not(sh_jac[1] == sh_err[0]):
        print('ERROR: m dimensions inconsistent, should be Jacobian [n x m]; error covariance [m x m]')
        print('Jacobian matrix dimensions')
        print(sh_jac)
        print('Error covariance matrix dimensions')
        print(sh_err)
        return -1, -1, -1, -1
        
    #section to generate model derived error -------------------------------------------------------
    
    if len(model_error) > 0:
        me=model_error
        jac_me=model_error_jacobian
        
        ln_me=np.shape(me)
        errme_2d = np.zeros((ln_me[0], ln_me[0]))
        np.fill_diagonal(errme_2d, me)
        err_me=errme_2d
    
        jac_me_t=np.transpose(jac_me)      
    
        JacmetMeJacme = np.matmul(jac_me_t,np.matmul(err_me,jac_me))
        err = err + JacmetMeJacme
    
        #perform inverse and matrix multiplication calculations ----------------------------------------
    jac_t=np.transpose(jac) #transpose of Jacobian (KT)
    
    try: 
        err_i=np.linalg.inv(err) #inverse of error covariance matrix (Se-1)
    except:
        print("ERROR: problem inverting error covariance matrix")
        return -1, -1, -1, -1
    
    try: 
        ap_i=np.linalg.inv(ap) #inverse of a priori error covariance matrix
    except:
        print("ERROR: problem inverting a priori covariance matrix")
        return -1, -1, -1, -1

    KtSK = np.matmul(jac,np.matmul(err_i,jac_t)) #calcuates KT Se-1 K

    try: 
        S_hat = np.linalg.inv(KtSK+ap_i) #calculate the inverse of (above + Sa-1)
    except:
        print("ERROR: problem inverting retrieval error covariance matrix")
        return -1, -1, -1, -1
    
    SIC = 0.5*np.log(np.linalg.det(np.matmul((KtSK+ap_i),ap))) #calculate Shannon Information Content    
    AvgK = np.matmul(S_hat,KtSK) #averaging kernel
    DFS = np.trace(AvgK) #degrees of freedom for signal (DFS) which is trace of averaging kernel
    
    return S_hat, SIC, AvgK, DFS  #returns retrieval error covariance matrix and the Shannon Information Content

In [3]:
#calculates the probability of detection given the parameter value (mu) and uncertainty (sigma)
#assumes PDF is gaussian normally distributed
def detect_prob(mu, sigma, doprint=0): 

    Pd = 1-0.5*(1+erf((-1*mu)/(sigma*np.sqrt(2))))  #detection probability, modified from CDF function

    Pd_pcnt_str=str(np.around(Pd*100,decimals=1))+'% positive probability' #string output version

    if doprint > 0:
        print(Pd_pcnt_str)

    return Pd, Pd_pcnt_str

In [4]:
#calculate detection probability metrics for a full range of fractional plastic coverage. 
#Also, return result at 95% or whatever is specified in fraction_threshold variable
def detect_prob_all(plastic_uncertainty,plastic_fraction,fraction_threshold=0.95):    

    #make array of values to assess (val) and dummy array to fill (det_prob)
    inc=np.arange(0, 10000, 1)
    val=inc/10000
    det_prob=np.arange(0, 10000, 1) / 10000
       
    #interpolate plastic_uncertainty to assessment values
    plastic_uncertainty_int = np.interp(val,plastic_fraction,plastic_uncertainty)    
    
    for x in inc:
        Pd, Pd_pcnt_str = detect_prob(val[x], plastic_uncertainty_int[x], doprint=0)
        det_prob[x] = Pd

    #get plastic fraction for a fraction_threshold detection probability 
    fraction_meeting_threshold = np.interp(fraction_threshold,det_prob,val)
    
    return det_prob, fraction_meeting_threshold
            

In [5]:
def print_out(S_hat, SIC, AvgK, DFS, jac, err, ap, me_err, numpts, params, me_params ):

    S_hat_diag=np.diagonal(S_hat)
    uncert=np.sqrt(S_hat_diag)

    np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
    print('Error covariance matrix:')
    print(S_hat)
    print()

    np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
    print('Averaging kernel matrix:')
    print(AvgK)
    print()
    np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
    print('Model Parameters:       ', params)
    print('Number of observations: ', numpts)
    print('A priori uncertainty:   ', np.sqrt(ap))
    print('Uncertainties:          ', uncert)
    print('Shannon Information Content:      ', SIC)
    print('Degrees of freedom for signal:    ', DFS)

## Section to read simulated dataset

In [6]:
#read netcdf4 file with simulation
#f = netCDF4.Dataset('simulations/Amir/plastics_toa_simulations_modisa_permutations.nc')
#f = netCDF4.Dataset('simulations/Amir/plastics_toa_simulations_modisa_permutations_v3.nc')
f = netCDF4.Dataset('./simulations/Amir/plastics_toa_simulations_modisa_permutations_v4.nc')

#read jacobian and data into jac and meas, respectively. Also get z (#cases,), m (measurent), n (param) lengths
jac_all=np.asarray(f.variables['K_Jac'])
meas=np.asarray(f.variables['rhot'])
param_order=np.asarray(f.variables['parameter'])
waveln=np.asarray(f.variables['wavelength'])
g=jac_all.shape
z_len=g[0]
m_len=g[1]
n_len=g[2]

#read the simulation specific parameters into a dataframe
df = pd.DataFrame({'Windspeed(m_s)': np.asarray(f.variables['Windspeed(m_s)']),
                   'Humidity(%)': np.asarray(f.variables['Humidity(%)']),
                   'FMF': np.asarray(f.variables['FMF']),
                   'AOD(869)': np.asarray(f.variables['AOD(869)']),
                   'chla(mg_m3)': np.asarray(f.variables['chla(mg_m3)']),
                   'plastic_fraction': np.asarray(f.variables['plastic_fraction']), 
                   'solz': np.asarray(f.variables['solz']),     
                   'relaz': np.asarray(f.variables['relaz']),  
                   'senz': np.asarray(f.variables['senz']),  
                  })

df["plastic_uncertainty"] = np.nan
df["SIC"] = np.nan
df["plastic_avgK"] = np.nan
df["DFS"] = np.nan

#close netcdf file
f.close()

In [7]:
#make a priori covariance matrix

#ap=(np.linspace(10.0,10.0,4))**2 #generate a priori error covariance matrix
WS_range=df['Windspeed(m_s)'].max() - df['Windspeed(m_s)'].min()
RH_range=df['Humidity(%)'].max() - df['Humidity(%)'].min()
FMF_range=df['FMF'].max() - df['FMF'].min()
AOD_range=df['AOD(869)'].max() - df['AOD(869)'].min()
CHL_range=df['chla(mg_m3)'].max() - df['chla(mg_m3)'].min()
PF_range=df['plastic_fraction'].max() - df['plastic_fraction'].min()


ap=np.asarray([WS_range,RH_range,FMF_range,AOD_range,CHL_range,PF_range]) #generate a priori error covariance matrix
ap=(ap/2)**2 #generate a priori error covariance matrix diagonals

In [8]:
#make error covariance matrix and prepare jacobian

rel_err = np.array([0.01587401, 0.01244266, 0.00938955, 0.01092051, 0.00302288,
    0.00544271, 0.00999068, 0.01467843, 0.0080387, 0.00944394,
    0.0193447, 0.0224503, 0.02386379])

for idx in range(0, z_len):

#for idx in range(29158, z_len):
#for idx in range(0, 1000):

    #rel_err=0.003  #relative error
    #sys_err=0.005
    #err=((meas[idx]*rel_err) + sys_err)**2 #generate error covariance matrix diagonals (code also takes 2d input)
        
    err=(meas[idx]*rel_err)**2
    
    #prepare jacobian
    this_jac=jac_all[idx]
    jac=this_jac.transpose()

    #check for NaN in jacobian
    if np.isnan(this_jac).any(): 
        print('Nan found in index ',idx)
    if np.isinf(this_jac).any(): 
        print('Inf found in index ',idx)
    
    #calculate rodgers stuff
    S_hat, SIC, AvgK, DFS = rodgers(jac, err, ap)

    df.loc[idx]["plastic_uncertainty"]=np.sqrt(S_hat[5,5]) 
    df.loc[idx]["SIC"]=SIC
    df.loc[idx]["plastic_avgK"]=AvgK[5,5]
    df.loc[idx]["DFS"]=DFS

df_orig=df.copy()    
    

In [12]:
#iterate through each set of conditions that have all the same parameter value except for plastic fraction. 
#create a new dataframe, and save one row for each set, with median values for plastic_uncertainty, SIC, plastic_avgK and DFS
#also, assess detection probability and save the plastic fraction value for 90% confident probability.

df.sort_values(by=['Windspeed(m_s)','Humidity(%)','FMF','AOD(869)','chla(mg_m3)','solz','relaz','senz'], inplace=True)

df.rename(columns={'plastic_fraction': 'plastic_threshold', 'plastic_uncertainty': 'plastic_unc_median', 'SIC': 'SIC_median', \
                   'plastic_avgK':'plastic_avgK_median', 'DFS': 'DFS_median'}, inplace=True)


fdf = df.copy()
fdfe = fdf[0:0]

#test case
#fdfe = fdf[0:1000]
#for idx in range(0, 1000, 10):

for idx in range(0, z_len-10, 10):
    if (idx % 1000) == 0:
        txt=str(idx)+' of '+str(z_len)
        print(txt)
    
    #get start and end points for this iteration
    st=idx
    ed=idx+10

    this=df.iloc[st:ed]

    this_plastic_fraction = np.asarray(df.iloc[st:ed]['plastic_threshold'])   
    this_plastic_uncertainty = np.asarray(df.iloc[st:ed]['plastic_unc_median'])   

    this_detect_prob, fraction_meeting_threshold = \
        detect_prob_all(this_plastic_uncertainty,this_plastic_fraction,fraction_threshold=0.95)

    fdfe = fdfe.append(df.iloc[st], ignore_index = True)
        
    fdfe.loc[fdfe.index[-1], 'plastic_threshold']= fraction_meeting_threshold
    
    #section to update field with median values for set
    fdfe.loc[fdfe.index[-1], 'plastic_unc_median']= np.median(this_plastic_uncertainty)
    fdfe.loc[fdfe.index[-1], 'SIC_median']= np.median(np.asarray(df.iloc[st:ed]['SIC_median']))
    fdfe.loc[fdfe.index[-1], 'plastic_avgK_median']= np.median(np.asarray(df.iloc[st:ed]['plastic_avgK_median']))
    fdfe.loc[fdfe.index[-1], 'DFS_median']= np.median(np.asarray(df.iloc[st:ed]['DFS_median']))
    


#change names in original df back
df.rename(columns={'plastic_threshold':'plastic_fraction', 'plastic_unc_median':'plastic_uncertainty', 'SIC_median':'SIC', \
                   'plastic_avgK_median':'plastic_avgK', 'DFS_median':'DFS'}, inplace=True)


fdfe

0 of 131220
1000 of 131220
2000 of 131220
3000 of 131220
4000 of 131220
5000 of 131220
6000 of 131220
7000 of 131220
8000 of 131220
9000 of 131220
10000 of 131220
11000 of 131220
12000 of 131220
13000 of 131220
14000 of 131220
15000 of 131220
16000 of 131220
17000 of 131220
18000 of 131220
19000 of 131220
20000 of 131220
21000 of 131220
22000 of 131220
23000 of 131220
24000 of 131220
25000 of 131220
26000 of 131220
27000 of 131220
28000 of 131220
29000 of 131220
30000 of 131220
31000 of 131220
32000 of 131220
33000 of 131220
34000 of 131220
35000 of 131220
36000 of 131220
37000 of 131220
38000 of 131220
39000 of 131220
40000 of 131220
41000 of 131220
42000 of 131220
43000 of 131220
44000 of 131220
45000 of 131220
46000 of 131220
47000 of 131220
48000 of 131220
49000 of 131220
50000 of 131220
51000 of 131220
52000 of 131220
53000 of 131220
54000 of 131220
55000 of 131220
56000 of 131220
57000 of 131220
58000 of 131220
59000 of 131220
60000 of 131220
61000 of 131220
62000 of 131220
63000

,Windspeed(m_s),Humidity(%),FMF,AOD(869),chla(mg_m3),plastic_threshold,solz,relaz,senz,plastic_unc_median,SIC_median,plastic_avgK_median,DFS_median
0,0.5,30.1,0.01,0.04,0.05,0.029137,15.0,40.0,15.0,0.017002,18.288161,0.998843,3.941760
1,0.5,30.1,0.01,0.04,0.05,0.007064,15.0,40.0,30.0,0.004577,16.612481,0.999916,3.960913
2,0.5,30.1,0.01,0.04,0.05,0.005750,15.0,40.0,60.0,0.003754,15.838545,0.999943,3.783445
3,0.5,30.1,0.01,0.04,0.05,0.006187,15.0,110.0,15.0,0.004024,17.041476,0.999935,3.937373
4,0.5,30.1,0.01,0.04,0.05,0.006523,15.0,110.0,30.0,0.004238,16.600650,0.999928,3.959462
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13116,10.0,94.9,0.95,0.30,2.00,0.009592,60.0,110.0,15.0,0.005840,10.817911,0.999864,3.113933
13117,10.0,94.9,0.95,0.30,2.00,0.012989,60.0,110.0,30.0,0.008115,10.465821,0.999737,2.870861
13118,10.0,94.9,0.95,0.30,2.00,0.019695,60.0,110.0,60.0,0.011919,9.901579,0.999432,3.043040
13119,10.0,94.9,0.95,0.30,2.00,0.010503,60.0,170.0,15.0,0.006410,11.091894,0.999836,3.291878


In [13]:
#save file or restart from here:

# save dataframe to pickle file
df.to_pickle('data/SQOOP_df_Amir_v4.pkl')
fdfe.to_pickle('data/SQOOP_fdfe_Amir_v4.pkl') 

